In [3]:
import tensorflow as tf
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [4]:
t1 = tf.constant([
[1,2,3],
[4,5,6],
[7,8,9],
],dtype=tf.float32)
ds1 = tf.data.Dataset.from_tensors(t1) 
ds2 = tf.data.Dataset.from_tensor_slices(t1)

2026-02-03 20:04:51.847849: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M5
2026-02-03 20:04:51.847889: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2026-02-03 20:04:51.847905: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
2026-02-03 20:04:51.847949: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-02-03 20:04:51.847975: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
for element in ds1:
    print(element)

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]], shape=(3, 3), dtype=float32)


2026-02-03 20:05:17.313427: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [6]:
for element in ds2:
    print(element)

tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32)
tf.Tensor([4. 5. 6.], shape=(3,), dtype=float32)
tf.Tensor([7. 8. 9.], shape=(3,), dtype=float32)


2026-02-03 20:05:33.156071: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [7]:
train, test = tf.keras.datasets.fashion_mnist.load_data() 
images, labels = train
images = images/255
type(images),type(labels)

(numpy.ndarray, numpy.ndarray)

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((images, labels)) 
dataset


<_TensorSliceDataset element_spec=(TensorSpec(shape=(28, 28), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))>

In [11]:
directory_url ="https://storage.googleapis.com/download.tensorflow.org/data/illiad/"
file_names = ['cowper.txt', 'derby.txt', 'butler.txt']
file_paths = [
tf.keras.utils.get_file(file_name, directory_url + file_name) 
for file_name in file_names
]

In [12]:
text_line_dataset = tf.data.TextLineDataset(file_paths)

In [13]:
for line in text_line_dataset.take(5):
    print(line.numpy())

b"\xef\xbb\xbfAchilles sing, O Goddess! Peleus' son;"
b'His wrath pernicious, who ten thousand woes'
b"Caused to Achaia's host, sent many a soul"
b'Illustrious into Ades premature,'
b'And Heroes gave (so stood the will of Jove)'


2026-02-03 20:06:53.432508: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [14]:
data_url ="https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"
dataset_dir = tf.keras.utils.get_file(
    origin=data_url,
    untar=True, 
    cache_dir= "stack_overflow",
    cache_subdir = ""
)
print(dataset_dir)
dataset_dir = pathlib.Path(dataset_dir)
train_dir = dataset_dir/'train'
train_dir

6053168/6053168 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
/tmp/.keras/stack_overflow_16k.tar.gz


PosixPath('/tmp/.keras/stack_overflow_16k.tar.gz/train')

In [15]:
batch_size = 32
seed = 42
raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [16]:
titanic_file = tf.keras.utils.get_file(
    "titanic.csv",
    "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
)

df = pd.read_csv(titanic_file)


In [17]:
import pandas as pd
import tensorflow as tf
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

numeric_cols = ['Age', 'Fare', 'SibSp', 'Parch']
df[numeric_cols] = df[numeric_cols].fillna(0)

categorical_cols = ['Sex', 'Cabin', 'Embarked', 'Pclass']  
df[categorical_cols] = df[categorical_cols].fillna('')
for col in categorical_cols:
    df[col] = df[col].astype('category').cat.codes
df = df.drop(columns=['Name', 'Ticket', 'PassengerId'])
titanic_dataset = tf.data.Dataset.from_tensor_slices(dict(df))
for example in titanic_dataset.take(1):
    print(example)


{'Survived': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'Pclass': <tf.Tensor: shape=(), dtype=int8, numpy=2>, 'Sex': <tf.Tensor: shape=(), dtype=int8, numpy=1>, 'Age': <tf.Tensor: shape=(), dtype=float64, numpy=22.0>, 'SibSp': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'Parch': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'Fare': <tf.Tensor: shape=(), dtype=float64, numpy=7.25>, 'Cabin': <tf.Tensor: shape=(), dtype=int16, numpy=0>, 'Embarked': <tf.Tensor: shape=(), dtype=int8, numpy=3>}


2026-02-03 20:07:59.956434: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [18]:
for feature_batch in titanic_dataset.take(1):
    for key,value in feature_batch.items():
        print(" {!r:20s}: {}".format(key,value))

 'Survived'          : 0
 'Pclass'            : 2
 'Sex'               : 1
 'Age'               : 22.0
 'SibSp'             : 1
 'Parch'             : 0
 'Fare'              : 7.25
 'Cabin'             : 0
 'Embarked'          : 3


2026-02-03 20:08:16.578031: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [19]:
import tensorflow as tf
import pandas as pd
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
local_file = tf.keras.utils.get_file("titanic.csv", url)
titanic_batches = tf.data.experimental.make_csv_dataset(
    local_file,                      
    batch_size=4,
    label_name="Survived",
    select_columns=['Pclass', 'Fare', 'Survived'],
    na_value="",                     
    num_epochs=1,
    shuffle=True
)

for feature_batch, label_batch in titanic_batches.take(1):
    print(f"Survived: {label_batch}")
    for key, value in feature_batch.items():
        print(f"{key:20s}: {value}")

Survived: [1 0 1 0]
Pclass              : [3 1 3 3]
Fare                : [ 11.2417 151.55    12.475    8.1375]


2026-02-03 20:08:29.746572: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [20]:
import pandas as pd
import tensorflow as tf
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
local_file = tf.keras.utils.get_file("titanic.csv", url)
df = pd.read_csv(local_file)
df[['Age', 'Fare', 'SibSp', 'Parch', 'Pclass', 'Survived']] = df[['Age', 'Fare', 'SibSp', 'Parch', 'Pclass', 'Survived']].fillna(0)
df[['Sex', 'Cabin', 'Embarked', 'Name', 'Ticket']] = df[['Sex', 'Cabin', 'Embarked', 'Name', 'Ticket']].fillna('')
for col in ['Sex', 'Embarked', 'Cabin']:
    df[col] = df[col].astype('category').cat.codes

df = df.drop(columns=['Name', 'Ticket', 'PassengerId'])
titanic_dataset = tf.data.Dataset.from_tensor_slices((dict(df.drop('Survived', axis=1)), df['Survived']))
titanic_dataset = titanic_dataset.batch(4)
for features, labels in titanic_dataset.take(1):
    print("Features:", features)
    print("Labels:", labels)


Features: {'Pclass': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([3, 1, 3, 1])>, 'Sex': <tf.Tensor: shape=(4,), dtype=int8, numpy=array([1, 0, 0, 0], dtype=int8)>, 'Age': <tf.Tensor: shape=(4,), dtype=float64, numpy=array([22., 38., 26., 35.])>, 'SibSp': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([1, 1, 0, 1])>, 'Parch': <tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 0, 0, 0])>, 'Fare': <tf.Tensor: shape=(4,), dtype=float64, numpy=array([ 7.25  , 71.2833,  7.925 , 53.1   ])>, 'Cabin': <tf.Tensor: shape=(4,), dtype=int16, numpy=array([ 0, 82,  0, 56], dtype=int16)>, 'Embarked': <tf.Tensor: shape=(4,), dtype=int8, numpy=array([3, 1, 3, 3], dtype=int8)>}
Labels: tf.Tensor([0 1 1 1], shape=(4,), dtype=int64)


2026-02-03 20:08:45.962857: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [22]:
flowers_root = tf.keras.utils.get_file(
'flower_photos',
"https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz",
untar=True
)
flowers_root = pathlib.Path(flowers_root)

In [23]:
for item in flowers_root.glob("*"):
    print(item)

/Users/anu/.keras/datasets/flower_photos/flower_photos


In [3]:
file_path_ds = tf.data.Dataset.list_files(str(flowers_root/'*/*'))

In [4]:
def process_path(file_path):
    label = tf.strings.split(file_path,os.sep)[-2]
    return tf.io.read_file(file_path),label

In [5]:
labeled_ds = file_path_ds.map(process_path)

In [6]:
for image_raw, label in labeled_ds.take(1):
    print(image_raw,label,sep="\n")

2026-02-03 20:50:08.992776: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : FAILED_PRECONDITION: /Users/anu/.keras/datasets/flower_photos/flower_photos/sunflowers; Is a directory
2026-02-03 20:50:08.992816: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : FAILED_PRECONDITION: /Users/anu/.keras/datasets/flower_photos/flower_photos/dandelion; Is a directory
2026-02-03 20:50:08.992834: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : FAILED_PRECONDITION: /Users/anu/.keras/datasets/flower_photos/flower_photos/tulips; Is a directory
2026-02-03 20:50:08.992840: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : FAILED_PRECONDITION: /Users/anu/.keras/datasets/flower_photos/flower_photos/roses; Is a directory
2026-02-03 20:50:08.992853: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed 

FailedPreconditionError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} /Users/anu/.keras/datasets/flower_photos/flower_photos/daisy; Is a directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext] name: 

In [8]:
inc_dataset = tf.data.Dataset.range(100)
dec_dataset = tf.data.Dataset.range(0, -100, -1)
dataset = tf.data.Dataset.zip((inc_dataset, dec_dataset))
batched_dataset = dataset.batch(4) 
for batch in batched_dataset.take(4):
    print([arr.numpy() for arr in batch])

[array([0, 1, 2, 3]), array([ 0, -1, -2, -3])]
[array([4, 5, 6, 7]), array([-4, -5, -6, -7])]
[array([ 8,  9, 10, 11]), array([ -8,  -9, -10, -11])]
[array([12, 13, 14, 15]), array([-12, -13, -14, -15])]


2026-02-03 20:52:44.210301: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [9]:
batched_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [10]:
dataset.batch(4,drop_remainder = True)

<_BatchDataset element_spec=(TensorSpec(shape=(4,), dtype=tf.int64, name=None), TensorSpec(shape=(4,), dtype=tf.int64, name=None))>

In [11]:
dataset = tf.data.Dataset.range(100)
dataset = dataset.map(lambda x: tf.fill([tf.cast(x, tf.int32)], x))
padded_batch_dataset = dataset.padded_batch(4, padded_shapes=(None,)) 
for batch in padded_batch_dataset.take(2):
    print(batch.numpy())
    print()

[[0 0 0]
 [1 0 0]
 [2 2 0]
 [3 3 3]]

[[4 4 4 4 0 0 0]
 [5 5 5 5 5 0 0]
 [6 6 6 6 6 6 0]
 [7 7 7 7 7 7 7]]



2026-02-03 20:53:50.385635: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:
dataset = tf.data.TextLineDataset(titanic_file)
dataset.shuffle(buffer_size=10)

NameError: name 'titanic_file' is not defined

In [13]:
import tensorflow as tf
import pandas as pd

titanic_file = tf.keras.utils.get_file(
    "titanic.csv",
    "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
)

dataset = tf.data.TextLineDataset(titanic_file)
dataset = dataset.shuffle(buffer_size=10)

for line in dataset.take(5):
    print(line.numpy())


b'PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked'
b'5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,,S'
b'9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,,S'
b'2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C'
b'3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S'


2026-02-03 20:55:27.828203: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [15]:
file_path_ds = tf.data.Dataset.list_files(str(flowers_root/'*/*'))

In [16]:
def parse_image(filename):
    label = tf.strings.split(filename, os.sep)[-2]
    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [128, 128])
    return image, label

In [17]:
image_ds = file_path_ds.map(parse_image)

In [19]:
def show(image,label):
    plt.imshow(image)
    plt.title(label.numpy().decode("utf-8"))
    plt.axis("off")
    plt.show()

In [20]:
for image,label in image_ds.take(3):
    show(image,label)

2026-02-03 20:59:24.546507: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : FAILED_PRECONDITION: /Users/anu/.keras/datasets/flower_photos/flower_photos/dandelion; Is a directory
2026-02-03 20:59:24.546598: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : FAILED_PRECONDITION: /Users/anu/.keras/datasets/flower_photos/flower_photos/roses; Is a directory
2026-02-03 20:59:24.546613: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : FAILED_PRECONDITION: /Users/anu/.keras/datasets/flower_photos/flower_photos/tulips; Is a directory
2026-02-03 20:59:24.546624: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : FAILED_PRECONDITION: /Users/anu/.keras/datasets/flower_photos/flower_photos/sunflowers; Is a directory
2026-02-03 20:59:24.546633: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed 

FailedPreconditionError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} /Users/anu/.keras/datasets/flower_photos/flower_photos/tulips; Is a directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext] name: 

In [21]:
import tensorflow as tf
import pathlib
import os
import matplotlib.pyplot as plt

# Dataset location
flowers_root = tf.keras.utils.get_file(
    'flower_photos',
    "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz",
    untar=True
)
flowers_root = pathlib.Path(flowers_root)

# ✅ Only image files (not directories)
file_path_ds = tf.data.Dataset.list_files(str(flowers_root / '*/*'))

# Process image path
def process_path(file_path):
    label = tf.strings.split(file_path, os.sep)[-2]
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [180, 180])
    image = image / 255.0
    return image, label

image_ds = file_path_ds.map(process_path)

# Test
for image, label in image_ds.take(3):
    print(image.shape, label.numpy())


2026-02-03 21:00:40.774245: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : FAILED_PRECONDITION: /Users/anu/.keras/datasets/flower_photos/flower_photos/dandelion; Is a directory
2026-02-03 21:00:40.774259: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : FAILED_PRECONDITION: /Users/anu/.keras/datasets/flower_photos/flower_photos/daisy; Is a directory
2026-02-03 21:00:40.774282: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : FAILED_PRECONDITION: /Users/anu/.keras/datasets/flower_photos/flower_photos/sunflowers; Is a directory
2026-02-03 21:00:40.774297: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at whole_file_read_ops.cc:116 : FAILED_PRECONDITION: /Users/anu/.keras/datasets/flower_photos/flower_photos/tulips; Is a directory
2026-02-03 21:00:40.774334: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed 

FailedPreconditionError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} /Users/anu/.keras/datasets/flower_photos/flower_photos/roses; Is a directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext] name: 

In [22]:
train, test = tf.keras.datasets.fashion_mnist.load_data()
images, labels = train
images = images/255.0
labels = labels.astype(np.int32)

NameError: name 'np' is not defined

In [23]:
fmnist_train_ds = tf.data.Dataset.from_tensor_slices((images, labels))
fmnist_train_ds = fmnist_train_ds.shuffle(5000).batch(32)
model = tf.keras.Sequential([
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(10)
])
model.compile(optimizer='adam',
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
metrics=['accuracy'])

In [24]:
model.fit(fmnist_train_ds, epochs=2)

Epoch 1/2


2026-02-03 21:02:46.698660: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7382 - loss: 0.7800
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8397 - loss: 0.4715


In [25]:
loss, accuracy = model.evaluate(fmnist_train_ds)
print("Loss :", loss)
print("Accuracy :", accuracy)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8508 - loss: 0.4367
Loss : 0.43517598509788513
Accuracy : 0.8509833216667175


In [26]:
predict_ds = tf.data.Dataset.from_tensor_slices(images).batch(32) 
result = model.predict(predict_ds, steps = 10)
print(result.shape)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
(320, 10)
